# Układy równań liniowych

## Zadanie 1

In [1]:
#include <cmath>
#include "aghMatrix.h"

In [2]:
// Addition of two matrices                                                                                                                                                   
template<typename T>
AGHMatrix<T> AGHMatrix<T>::operator+(const AGHMatrix<T>& rhs) 
{
  if (rows != rhs.get_rows() || cols != rhs.get_cols())
  {
    throw std::runtime_error("Matrices have to be of same size");
  }

  AGHMatrix<T> mat(rows, cols, 0);

  for (int i = 0; i < rows; i++) {
    for (int j = 0; j < cols; j++) {
      mat(i, j) = this->matrix[i][j] + rhs(i, j);
    }
  }

  return mat;
}

// Left multiplication of this matrix and another                                                                                                                              
template<typename T>
AGHMatrix<T> AGHMatrix<T>::operator*(const AGHMatrix<T>& rhs) 
{
  if (cols != rhs.get_rows()) {
    throw std::runtime_error("Incorrect matrix dimensions");
  }

  AGHMatrix<T> mat(rows, rhs.get_cols(), 0);

  for (int i = 0; i < rows; i++) {
    for (int j = 0; j < rhs.get_cols(); j++) {
      T sum = 0;
      for (int k = 0; k < cols; k++) {
        sum += this->matrix[i][k] * rhs(k, j);
      }
      mat(i, j) = sum;
    }
  }

  return mat;
}

## Zadanie 2

Aby dokonać transpozycji macierzy zamieniamy kolumny z wierszami.
Macierz jest symetryczna wtw. gdy jest równa swojej transpozycji.

In [3]:
template<typename T>
AGHMatrix<T> AGHMatrix<T>::transpose()
{
  AGHMatrix<T> mat(cols, rows, 0);

  for (int i = 0; i < cols; i++) {
    for (int j = 0; j < rows; j++) {
      mat(i, j) = this->matrix[j][i];
    }
  }

  return mat;
}

template<typename T>
bool AGHMatrix<T>::is_symmetric()
{
  return (rows == cols && *this == transpose());
}

Wyznaczik macierzy liczę dopowadzając macierz do postaci schodkowej za pomocą metody gaussa. Implementacja znajduje się poniżej.

## Zadanie 3 - Faktoryzacja w metodzie LU

Faktoryzację LU dokonuję za pomocą metody Doolittle'a. Równanie A = LU traktujemy jako układ n^2 równań z n^2 niewiadomymi. Dodatkowo zakładamy, że na przekątej głównej macierzy L znajdują się jedynki (oczywiście z założenia metody LU L i U to są macierze trójkątne, odpowiednio dolne i górne). Przy tych założeniach otrzymujemy zależności na wartości elementów macierzy L i U, które są dane wzorami rekurencyjnymi (zaimplementowano ponieżej).

In [4]:
template<typename T>
std::pair<AGHMatrix<double>, AGHMatrix<double>> doolittleDecomposition(AGHMatrix<T> mat) {
    if (mat.get_rows() != mat.get_cols()) {
        throw std::runtime_error("Matrix has to be sqare");
    }

    int n = mat.get_rows();
    AGHMatrix<T> u(n, n, 0);
    AGHMatrix<T> l(n, n, 0);
    for (int i = 0; i < n; i++) l(i, i) = 1;

    for (int i = 0; i < n; i++) {
        for (int j = i; j < n; j++) {
            T s = 0;
            for (int k = 0; k < i; k++) {
                s += l(i, k) * u(k, j);
            }
            u(i, j) = mat(i, j) - s;
        }

        for (int j = i + 1; j < n; j++) {
            T s = 0;
            for (int k = 0; k < i; k++) {
                s += l(j, k) * u(k, i);
            }

            if (u(i, i) == 0) {
                throw std::runtime_error("Division by 0");
            }

            l(j, i) = (1.0 / u(i, i)) * (mat(j, i) - s);
        }
    }

    return std::make_pair(l, u);
}

In [5]:
AGHMatrix<double> mat({{ 5.0, 3.0, 2.0 }, 
                            { 1.0, 2.0, 0.0 }, 
                            { 3.0, 0.0, 4.0 }});

auto res = doolittleDecomposition(mat);
std::cout << res.first;
std::cout << res.second;

1, 0, 0, 
0.2, 1, 0, 
0.6, -1.28571, 1, 

5, 3, 2, 
0, 1.4, -0.4, 
0, 0, 2.28571, 



Powyżej jest widoczna macierz L i macierz U (mat = LU)

## Zadanie 4 - Faktoryzacja Cholesky'ego

Dodatnio określoną symetryczną macierz rozkładamy na macierz trójkątną dolną i jej transpozycję (A = LL^T). Po rozpisaniu mnożenia macierzy otrzymujemy szereg równań rekurencyjnych (rozwiązujemy od pierwszego równania, gdzie mamy tylko jedną niewiadomą).

In [6]:
template<typename T>
AGHMatrix<double> choleskyDecomposition(AGHMatrix<T> mat) {
    if (!mat.is_symmetric()) {
        throw std::runtime_error("Matrix is not symmetric");
    }
    
    int n = mat.get_rows();
    AGHMatrix<double> l(n, n, 0);

    for (int i = 0; i < n; i++) {
        for (int j = 0; j <= i; j++) {
            if (i == j) {
                T s = 0;
                for (int k = 0; k < i; k++) {
                    s += l(i, k) * l(i, k);
                }
                l(i, j) = sqrt(mat(i, j) - s);
            } else {
                T s = 0;
                for (int k = 0; k < i; k++) {
                    s += l(j, k) * l(i, k);
                }
                l(i, j) = (mat(i, j) - s) / l(j, j);
            }
        }
    }

    return l;
}

In [7]:
AGHMatrix<double> mat2({{ 4.0, 12.0, -16.0 }, 
                            { 12.0, 37.0, -43.0 }, 
                            { -16.0, -43.0, 98.0 }});

AGHMatrix<double> L = choleskyDecomposition(mat2); 
std::cout << L << std::endl;
std::cout << L.transpose() << std::endl;

2, 0, 0, 
6, 1, 0, 
-8, 5, 3, 


2, 6, -8, 
0, 1, 5, 
0, 0, 3, 




### Porównanie faktoryzacji Doolittle'a i Choleskiego

W metodzie LU podstawowym problemem jest faktoryzacja macierzy na dwie macierze trójkątne - jedna jest
macierzą trójkątną górną, a druga dolną. Aby dokonać faktoryzacji korzystamy z metody Doolittla, w której
iteracyjnie (ze złożonością O(n^3)) obliczamy kolejne wartości w macierzach rozkładu. Dodatkowo, metoda działa tylko jeżeli  w macierzy trójkątniej górnej na przekątnej nie występuje 0 (w takiej sytuacji możemy spróbować zamienić dany rząd z innym - https://en.wikipedia.org/wiki/LU_decomposition). 

W metodzie Choleskiego
postępujemy podobnie, ale mamy dodatkowe ograniczenia, mianowicie rozkładana macierz musi być symetryczna
i dodatnio określona. Również macierz rozkładamy na macierz trójkątną górną i dolną, ale druga macierz jest transpozycją pierwszej. Złożoność obliczeniowa jest również rzędu O(n^3).

## Zadanie 5 - Eliminacja Gaussa

Eliminacja Gaussa polega na sprowadzeniu macierzy do postaci schodkowej. Dokonujemy tego poprzez wynonanie szeregu  operacji na wierszach macierzy (dodawanie kombinacji liniowych elementów innych wierszy macierzy).

In [8]:
enum SystemSolution { INF, ONE, ZERO };

In [9]:
template<typename T>
SystemSolution to_echelon_form(AGHMatrix<T> &mat) {
    int n = mat.get_rows();

    // powtarzamy redukcję dla każdej niewiadomej
    for (int k = 0; k < n; k++) {
        // znajdujemy wiersz, w której współ. przy k-tej niewiadomej ma największą wartość bezwzględną
        // moglibyśmy wybrać dowolny wiersz, w którym współ. przy k-tej niewiadomej ma niezerową wartość,
        // ale wybranie wartości z największą wartością bezwzględną sprawia, że algorytm jest bardziej 
        // stabilny numerycznie.
        int mx = k;
        for (int i = k; i < n; i++) {
            if (std::abs(mat(i, k)) > std::abs(mat(mx, k))) {
                mx = i;
            }
        }

        // znaleziony wiersz zamieniamy z aktualnym
        mat.swap_rows(k, mx);

        // maksymalna wartość współ. przy k-tej niewiadomej jest równa zero
        // (z tego wynika, że wszystkie są równe zero)
        if (mat(k, k) == 0) {
            // z Tw. Kroneckera-Capellego
            return mat(k, n) != 0 ? ZERO : INF;
        }

        // dla wierszy niżej
        for (int i = k + 1; i < n; i++) {
            // wartość, przez którą będziemy mnożyć k-ty wiersz, aby wyzerować k-ty współczynnik
            // w i-tym wierszu
            double f = mat(i, k) / mat(k, k);

            for (int j = k + 1; j < mat.get_cols(); j++) {
                // od każdego współczynnika odejmujemy k-ty wiersz pomnożony przez f
                mat(i, j) -= mat(k, j) * f;
            }

            // zerujemy współczynnik (robimy to poza pętlą aby uniknąć uzyskania małej liczby zmiennoprzecinkowej w
            // wyniku niedokładności)
            mat(i, k) = 0;
        }
    }

    return ONE;
}

template<typename T>
AGHMatrix<T> get_res(AGHMatrix<T> echelonMat) {
    int n = echelonMat.get_rows();
    AGHMatrix<T> res(n, 1, 0);

    // zaczynamy od końca (mamy jedną niewiadomą)
    for (int k = n - 1; k >= 0; k--) {
        // sumujemy kombinacje poprzednio obliczonych niewiadomych
        T s = 0;
        for (int i = k + 1; i < n; i++) {
            s += echelonMat(k, i) * res(i, 0);
        }

        // obliczamy wartość k-tej niewiadomej
        res(k, 0) = (echelonMat(k, n) - s) / echelonMat(k, k);
    }

    return res.transpose();
}

In [10]:
AGHMatrix<double> mat3({{ 2.0, 1.0, -1.0, 8.0 }, 
                            { -3.0, -1.0, 2.0, -11.0 }, 
                            { -2.0, 1.0, 2.0, -3.0 }});

std::cout << to_echelon_form(mat3) << std::endl;
std::cout << mat3 << std::endl;
std::cout << get_res(mat3);

1
-3, -1, 2, -11, 
0, 1.66667, 0.666667, 4.33333, 
0, 0, 0.2, -0.2, 


2, 3, -1, 



Powyżej widzimy postać schodkową macierzy *mat3* oraz rozwiązanie układu równań.

### Wyzancznik macierzy

Wyznaczik macierzy schodkowej jest równy iloczynowi wartości leżących na przekątnej macierzy.

In [16]:
template <typename T>
double det(AGHMatrix<T> mat) {
    if (mat.get_rows() != mat.get_cols()) {
        throw std::runtime_error("Matrix must be square");
    }

    to_echelon_form(mat);
    double d = mat(0, 0);

    for (int i = 1; i < mat.get_rows(); i++) {
        d *= mat(i, i);
    }

    return d;
}

## Zadanie 6 - Metoda Jacobiego

In [11]:
template<typename T>
bool is_close_enough(AGHMatrix<T> prev, AGHMatrix<T> curr, double eps) {
    for (int i = 0; i < prev.get_rows(); i++) {
        if (std::abs(prev(i, 0) - curr(i, 0)) > eps) {
            return false;
        }
    }

    return true;
}

template<typename T>
AGHMatrix<T> jacobi(AGHMatrix<T> a, AGHMatrix<T> b, double eps) {
    int n = a.get_rows();
    AGHMatrix<T> x(n, 1, 0);
    AGHMatrix<T> last(n, 1, 0);
    AGHMatrix<T> L(n, n, 0);
    AGHMatrix<T> D(n, n, 0);
    AGHMatrix<T> U(n, n, 0);
    
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < i; j++) L(i, j) = a(i, j);
        for (int j = n - 1; j > i; j--) U(i, j) = a(i, j);
        D(i, i) = 1.0 / a(i, i);
    }

    AGHMatrix<T> M = (L + U) * (-1);

    for (;;) {
        AGHMatrix<T> tmp = D * ((M * x) + b);
        if (is_close_enough(x, tmp, eps)) break;
        x = tmp;
    }

    return x;
}

In [12]:
AGHMatrix<double> a({{ 4.0, -1.0, -0.2, 2.0 }, 
                            { -1.0, 5.0, 0, -2.0 }, 
                            { 0.2, 1.0, 10.0 -1.0 },
                            { 0, -2.0, -1.0, 4.0 }});
AGHMatrix<double> b({{ 30.0, 0, -10.0, 5.0 }});

In [15]:
std::cout << jacobi(a, b.transpose(), 0.000001);

6.98818, 
2.18342, 
-1.50901, 
1.96446, 



Metoda Jacobiego działa w inny sposób od powyższych algorytmów, mianowicie najpierw rozkładamy daną macierz na sumę trzech macierzy - trójkątnej górnej, dolnej i macierzy diagonalnej, a następnie po odpowiednich przekształceniach oblicza iteracyjnie wartości macierzy kolumnowej x, zaczynając od jakiegoś danego stanu początkowego. Kolejne wartości macierzy kolumnowej x zbiegają do poszukiwanego rozwiązania.

Mamy również dodatkowe ograniczenie, mianowicie macierz współczynników musi być macierzą przekątniowo dominująca, co oznacza, że suma wartości bezwzględnych elementów nieleżących na przekątnej musi być mniejsza bądź równa wartości bezwzględej elementu leżącego na przekątnej (w danym wierszu).